In [5]:
!pip install pandas pyarrow numpy scikit-learn statsmodels matplotlib tqdm


  Using cached scikit_learn-1.7.2-cp312-cp312-win_amd64.whl.metadata (11 kB)
Using cached scikit_learn-1.7.2-cp312-cp312-win_amd64.whl (8.7 MB)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import shutil, pathlib, os

src = pathlib.Path("air+quality/AirQualityUCI.csv")  # 你现在的路径
dst = pathlib.Path("data_raw/AirQualityUCI.csv")     # 我们的标准输入路径
dst.parent.mkdir(parents=True, exist_ok=True)
shutil.copy2(src, dst)

print("✅ Copied to:", dst.resolve())
print("Here:", os.listdir("data_raw"))


✅ Copied to: C:\Users\10305\Downloads\air+quality\data_raw\AirQualityUCI.csv
Here: ['AirQualityUCI.csv']


In [9]:
import pandas as pd
df_raw = pd.read_csv("data_raw/AirQualityUCI.csv", sep=";", decimal=",")
df_raw.head(3), df_raw.shape


(         Date      Time  CO(GT)  PT08.S1(CO)  NMHC(GT)  C6H6(GT)  \
 0  10/03/2004  18.00.00     2.6       1360.0     150.0      11.9   
 1  10/03/2004  19.00.00     2.0       1292.0     112.0       9.4   
 2  10/03/2004  20.00.00     2.2       1402.0      88.0       9.0   
 
    PT08.S2(NMHC)  NOx(GT)  PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)  \
 0         1046.0    166.0        1056.0    113.0        1692.0       1268.0   
 1          955.0    103.0        1174.0     92.0        1559.0        972.0   
 2          939.0    131.0        1140.0    114.0        1555.0       1074.0   
 
       T    RH      AH  Unnamed: 15  Unnamed: 16  
 0  13.6  48.9  0.7578          NaN          NaN  
 1  13.3  47.7  0.7255          NaN          NaN  
 2  11.9  54.0  0.7502          NaN          NaN  ,
 (9471, 17))

In [16]:
import pandas as pd
import numpy as np

# —— 从 df_raw 重新开始 —— 
df = df_raw.loc[:, ~df_raw.columns.str.contains('^Unnamed')].copy()

# 1) 修正 Time: '18.00.00' -> '18:00:00'
time_fixed = (
    df["Time"].astype(str)
      .str.replace('.', ':', regex=False)
      .str.strip()
)

# 2) 合并 Date + 修正后的 Time，并显式指定格式
#    Date 是日/月/年；Time 是时:分:秒
df["timestamp"] = pd.to_datetime(
    df["Date"].astype(str).str.strip() + " " + time_fixed,
    format="%d/%m/%Y %H:%M:%S",    # 关键
    errors="coerce"
)

# 3) 清理并设为索引
df = df.drop(columns=["Date", "Time"])
df = df.sort_values("timestamp")
na_ts = df["timestamp"].isna().sum()
print("NaT in timestamp:", na_ts)

df = df.dropna(subset=["timestamp"])
df = df.drop_duplicates("timestamp").set_index("timestamp")

# 4) -200 -> NaN；统一数值类型
df = df.replace(-200, np.nan)
for c in df.columns:
    df[c] = pd.to_numeric(df[c], errors="coerce")

print("shape after basic clean:", df.shape)
df.head(3)


NaT in timestamp: 114
shape after basic clean: (9357, 13)


,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
timestamp,,,,,,,,,,,,,
2004-03-10 18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
2004-03-10 19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2004-03-10 20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502


In [18]:
# 按小时对齐 -> 时间插值 -> 滚动均值兜底
df = df.asfreq("H")  # 若有缺小时，会补出空行

df_imp = df.interpolate(method="time", limit_direction="both")
df_imp = df_imp.fillna(df_imp.rolling(window=3, min_periods=1).mean())

missing_total = int(df_imp.isna().sum().sum())
print("Total missing after imputation:", missing_total)
df_imp.head(3)


Total missing after imputation: 0


C:\Users\10305\AppData\Local\Temp\ipykernel_3580\3225207664.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.asfreq("H")  # 若有缺小时，会补出空行


,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
timestamp,,,,,,,,,,,,,
2004-03-10 18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
2004-03-10 19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2004-03-10 20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502


In [20]:
import statsmodels.api as sm
import pandas as pd

def mark_anomalies_stl(df_in, cols=None, z=3.0):
    if cols is None:
        cols = [c for c in df_in.columns if any(k in c.lower() for k in ["co","nox","no2","c6h6","nmhc"])]
        if not cols: cols = df_in.columns.tolist()
    out = df_in.copy()
    flag = pd.Series(0, index=out.index, dtype="int8")
    for c in cols:
        s = out[c].dropna()
        if len(s) < 48: 
            continue
        try:
            res = sm.tsa.STL(s, period=24, robust=True).fit()
            resid = s - (res.trend + res.seasonal)
            zscore = (resid - resid.mean()) / (resid.std(ddof=1) + 1e-9)
            f = (zscore.abs() > z).reindex(out.index).fillna(0).astype("int8")
            flag = flag | f
        except Exception:
            pass
    out["anomaly_flag"] = flag
    return out

df_clean = mark_anomalies_stl(df_imp, z=3.0)
df_clean["anomaly_flag"].value_counts()


anomaly_flag
0    8454
1     903
Name: count, dtype: int64

In [22]:
from pathlib import Path
Path("data_artifacts").mkdir(parents=True, exist_ok=True)
clean_path = "data_artifacts/clean_air_quality.parquet"
df_clean.to_parquet(clean_path)
print("✅ saved:", clean_path, df_clean.shape)


✅ saved: data_artifacts/clean_air_quality.parquet (9357, 14)


In [24]:
import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from pathlib import Path

def add_calendar(df):
    out = df.copy()
    out["hour"] = out.index.hour.astype("int8")
    out["weekday"] = out.index.weekday.astype("int8")
    out["month"] = out.index.month.astype("int8")
    return out

def add_lags(df, cols=None, lags=(1,2,3,6,12,24)):
    out = df.copy()
    if cols is None:
        cols = [c for c in df.columns if df[c].dtype.kind in "fc" and c!="anomaly_flag"]
    for c in cols:
        for L in lags:
            out[f"{c}_lag{L}"] = out[c].shift(L)
    return out

def add_rollings(df, cols=None, windows=(3,6,12,24)):
    out = df.copy()
    if cols is None:
        cols = [c for c in df.columns if df[c].dtype.kind in "fc" and c!="anomaly_flag"]
    for c in cols:
        s = out[c]
        for w in windows:
            roll = s.rolling(window=w, min_periods=max(1, w//2))
            out[f"{c}_r{w}_mean"] = roll.mean()
            out[f"{c}_r{w}_std"]  = roll.std()
    return out

def shift_target(df, target_col="CO(GT)", horizon=1):
    out = df.copy()
    out[f"y_t+{horizon}"] = out[target_col].shift(-horizon)
    out[f"naive_yhat_t+{horizon}"] = out[target_col]
    bins = [-np.inf, 1.5, 2.5, np.inf]   # CO 三档阈值
    labels = ["low","mid","high"]
    out[f"co_level_t+{horizon}"] = pd.cut(out[f"y_t+{horizon}"], bins=bins, labels=labels)
    return out

def make_features(df, horizon=1):
    x = add_calendar(df)
    x = add_lags(x)
    x = add_rollings(x)
    x = shift_target(x, "CO(GT)", horizon)
    return x

def time_split(df, train_end="2004-12-31 23:00", valid_hours=168):
    df = df.sort_index()
    train = df.loc[:train_end].copy()
    test  = df.loc[train_end:].copy()
    if valid_hours and valid_hours > 0:
        valid = train.iloc[-valid_hours:].copy()
        train = train.iloc[:-valid_hours].copy()
    else:
        valid = df.iloc[0:0].copy()
    return train, valid, test

def fit_scaler(train, exclude_cols):
    num_cols = [c for c in train.columns if train[c].dtype.kind in "fc" and c not in exclude_cols]
    scaler = StandardScaler().fit(train[num_cols].fillna(0.0))
    return scaler, num_cols

def apply_scaler(df, scaler, num_cols):
    out = df.copy()
    out[num_cols] = scaler.transform(out[num_cols].fillna(0.0))
    return out

EXPORT = Path("data_artifacts"); (EXPORT/"splits").mkdir(parents=True, exist_ok=True)
for H in [1,6,12,24]:
    feats = make_features(df_clean, horizon=H).dropna(subset=[f"y_t+{H}"])
    feats.to_parquet(EXPORT/f"features_h+{H}.parquet")
    train, valid, test = time_split(feats, train_end="2004-12-31 23:00", valid_hours=168)
    y_cols = [f"y_t+{H}", f"naive_yhat_t+{H}"]
    scaler, num_cols = fit_scaler(train, exclude_cols=y_cols)
    train_s = apply_scaler(train, scaler, num_cols)
    valid_s = apply_scaler(valid, scaler, num_cols) if len(valid) else valid
    test_s  = apply_scaler(test,  scaler, num_cols)
    outdir = EXPORT/"splits"/f"h{H}"; outdir.mkdir(parents=True, exist_ok=True)
    train_s.to_parquet(outdir/"train.parquet")
    if len(valid_s): valid_s.to_parquet(outdir/"valid.parquet")
    test_s.to_parquet(outdir/"test.parquet")
    print(f"✅ h={H} saved:",
          EXPORT/f"features_h+{H}.parquet",
          outdir/"train.parquet",
          outdir/"valid.parquet",
          outdir/"test.parquet", sep="\n- ")
print("🎉 Done.")


C:\Users\10305\AppData\Local\Temp\ipykernel_3580\653758270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{c}_r{w}_mean"] = roll.mean()
C:\Users\10305\AppData\Local\Temp\ipykernel_3580\653758270.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{c}_r{w}_std"]  = roll.std()
C:\Users\10305\AppData\Local\Temp\ipykernel_3580\653758270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

✅ h=1 saved:
- data_artifacts\features_h+1.parquet
- data_artifacts\splits\h1\train.parquet
- data_artifacts\splits\h1\valid.parquet
- data_artifacts\splits\h1\test.parquet


C:\Users\10305\AppData\Local\Temp\ipykernel_3580\653758270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{c}_r{w}_mean"] = roll.mean()
C:\Users\10305\AppData\Local\Temp\ipykernel_3580\653758270.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{c}_r{w}_std"]  = roll.std()
C:\Users\10305\AppData\Local\Temp\ipykernel_3580\653758270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

✅ h=6 saved:
- data_artifacts\features_h+6.parquet
- data_artifacts\splits\h6\train.parquet
- data_artifacts\splits\h6\valid.parquet
- data_artifacts\splits\h6\test.parquet


C:\Users\10305\AppData\Local\Temp\ipykernel_3580\653758270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{c}_r{w}_mean"] = roll.mean()
C:\Users\10305\AppData\Local\Temp\ipykernel_3580\653758270.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{c}_r{w}_std"]  = roll.std()
C:\Users\10305\AppData\Local\Temp\ipykernel_3580\653758270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

✅ h=12 saved:
- data_artifacts\features_h+12.parquet
- data_artifacts\splits\h12\train.parquet
- data_artifacts\splits\h12\valid.parquet
- data_artifacts\splits\h12\test.parquet


C:\Users\10305\AppData\Local\Temp\ipykernel_3580\653758270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{c}_r{w}_mean"] = roll.mean()
C:\Users\10305\AppData\Local\Temp\ipykernel_3580\653758270.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{c}_r{w}_std"]  = roll.std()
C:\Users\10305\AppData\Local\Temp\ipykernel_3580\653758270.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

✅ h=24 saved:
- data_artifacts\features_h+24.parquet
- data_artifacts\splits\h24\train.parquet
- data_artifacts\splits\h24\valid.parquet
- data_artifacts\splits\h24\test.parquet
🎉 Done.


In [26]:
import pandas as pd, glob, os
# 1) 有哪些产物
sorted(glob.glob("data_artifacts/**/*.*", recursive=True))[:8]
# 2) 随机看一个分割的形状
pd.read_parquet("data_artifacts/splits/h1/train.parquet").shape
# 3) 时间范围 sanity check
dfh1 = pd.read_parquet("data_artifacts/features_h+1.parquet")
dfh1.index.min(), dfh1.index.max()


(Timestamp('2004-03-10 18:00:00'), Timestamp('2005-04-04 13:00:00'))

In [28]:
import pandas as pd
base = "data_artifacts"
feats = pd.read_parquet(f"{base}/features_h+24.parquet")
train = pd.read_parquet(f"{base}/splits/h24/train.parquet")
valid = pd.read_parquet(f"{base}/splits/h24/valid.parquet")
test  = pd.read_parquet(f"{base}/splits/h24/test.parquet")

feats.index.min(), feats.index.max(), feats.shape
train.index.min(), train.index.max(), train.shape
valid.index.min(), valid.index.max(), valid.shape
test.index.min(),  test.index.max(),  test.shape


(Timestamp('2004-12-31 23:00:00'),
 Timestamp('2005-04-03 14:00:00'),
 (2224, 826))

In [30]:
valid_start_end = (valid.index.min(), valid.index.max(), len(valid))
test_start_end  = (test.index.min(),  test.index.max(),  len(test))
valid_start_end, test_start_end


((Timestamp('2004-12-25 00:00:00'), Timestamp('2004-12-31 23:00:00'), 168),
 (Timestamp('2004-12-31 23:00:00'), Timestamp('2005-04-03 14:00:00'), 2224))